In [47]:
import random
import math

actions = ['POISON', 'FART_SPRAY', 'OXYCOTTON', 'EXERCISE']

def R(a):
    if a == 'POISON':
        return -100
    elif a == 'FART_SPRAY':
        return 0
    elif a == 'OXYCOTTON':
        return -10
    elif a == 'EXERCISE':
        return 50
    else:
        return 0

q = [(random.random()-0.5)*20 for _ in range(len(actions))]

In [68]:
from tqdm import trange
import time

# epsilon greedy method

epsilon = 0.1
cum_reward = 0
n_steps = 100
lr = 0.025

for i in (t:=trange(n_steps)):
    if random.random() > 0.1:
        highest_reward = -math.inf
        best_action = ''
        best_action_idx = -1
        for i in range(len(actions)):
            if R(actions[i]) > highest_reward:
                highest_reward = R(actions[i])
                best_action = actions[i]
                best_action_idx = i
        reward = R(best_action)
        cum_reward += reward
        q[best_action_idx] = (1-lr) * q[best_action_idx] + lr * reward

    else:
        random_action_idx = math.floor(random.random() * len(actions))
        random_action = actions[random_action_idx]
        reward = R(random_action)
        cum_reward += reward
        q[random_action_idx] = (1-lr) * q[best_action_idx] + lr * reward
    
    time.sleep(100)
    
    t.set_description(f'avg reward: {cum_reward/(i+1)}')




avg reward: 87362.5:   8%|▊         | 8103/100000 [00:03<00:44, 2053.70it/s]           


KeyboardInterrupt: 